In [14]:
#Deploying model
script_file = "crop_predict.py"

import json
import joblib
import numpy as np
from azureml.core.model import Model

# Called when the service is loaded
def init():
    global model
    # Get the path to the deployed model file and load it
    model_path = Model.get_model_path('model.pkl')
    model = joblib.load(model_path)

# Called when a request is received
def run(raw_data):
    # Get the input data as a numpy array
    data = np.array(json.loads(raw_data)['data'])
    # Get a prediction from the model
    predictions = model.predict(data)
    
    return json.dumps(predictions.tolist())

In [23]:
from azureml.core.model import Model
from azureml.core import Workspace

ws = Workspace(subscription_id="7b7e94ec-f305-433f-829b-475500f5b6b3",
               resource_group="KS_hack_2022",
               workspace_name="KS_reactor_hack")

# Register model
model = Model.register(ws, model_name="model", model_path="./model.pkl")

Registering model model


In [24]:
from azureml.core import Environment
from azureml.core.model import InferenceConfig

env = Environment(name="project_environment")
dummy_inference_config = InferenceConfig(
    environment=env,
    source_directory="./source_dir",
    entry_script="./echo_score.py",
)

In [25]:
from azureml.core.webservice import LocalWebservice

deployment_config = LocalWebservice.deploy_configuration(port=6789)

In [ ]:
service = Model.deploy(
    ws,
    "myservice",
    [model],
    dummy_inference_config,
    deployment_config,
    overwrite=True,
)
service.wait_for_deployment(show_output=True)

In [ ]:
print(service.get_logs())

In [21]:
from azureml.core.webservice import AciWebservice
from azure.ai.ml import MLClient
from azureml.core.model import InferenceConfig
from azure.ai.ml.entities import Environment, BuildContext
from azure.identity import DefaultAzureCredential

#Enter details of your AzureML workspace
subscription_id = '7b7e94ec-f305-433f-829b-475500f5b6b3'
resource_group = 'KS_hack_2022'
workspace = 'KS_reactor_hack'

#connect to the workspace
ml_client = MLClient(DefaultAzureCredential(), subscription_id, resource_group, workspace)

env_docker_conda = Environment(
    image="mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04",
    conda_file="environment.yml",
    name="docker-image-plus-conda-example",
    description="Environment created from a Docker image plus Conda environment.",
)
ml_client.environments.create_or_update(env_docker_conda)

# Configure the scoring environment
inference_config = InferenceConfig(entry_script=script_file,
                                   environment=env_docker_conda)

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)

service_name = "crop_predict-service"

service = Model.deploy(ws, service_name, [model], inference_config, deployment_config)

service.wait_for_deployment(True)
print(service.state)

AttributeError: 'Environment' object has no attribute 'python'

In [2]:
script_file = "/mnt/batch/tasks/shared/LS_root/mounts/clusters/rlsri23051/code/Users/rlsri2305/score_crop.py"

In [4]:
%%writefile $script_file
import json
import joblib
import numpy as np
from azureml.core.model import Model

# Called when the service is loaded
def init():
    global model
    # Get the path to the deployed model file and load it
    model_path = Model.get_model_path('model')
    model = joblib.load(model_path)

# Called when a request is received
def run(raw_data):
    # Get the input data as a numpy array
    data = np.array(json.loads(raw_data)['data'])
    # Get a prediction from the model
    predictions = model.predict(data)
    
    return json.dumps(predictions.tolist())

Overwriting /mnt/batch/tasks/shared/LS_root/mounts/clusters/rlsri23051/code/Users/rlsri2305/score_crop.py


In [16]:
from azureml.core.webservice import AciWebservice
from azureml.core.model import InferenceConfig
from azureml.core.model import Model
from azureml.core import Workspace
from azure.ai.ml.entities import Environment, BuildContext
#import required libraries
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential
from azure.ai.ml import command, Input

#Enter details of your AzureML workspace
subscription_id = '7b7e94ec-f305-433f-829b-475500f5b6b3'
resource_group = 'KS_hack_2022'
workspace = 'KS_reactor_hack'

#connect to the workspace
ml_client = MLClient(DefaultAzureCredential(), subscription_id, resource_group, workspace)

ws = Workspace(subscription_id="7b7e94ec-f305-433f-829b-475500f5b6b3",
               resource_group="KS_hack_2022",
               workspace_name="KS_reactor_hack")

env = Environment(
    image="mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04",
    python="azureml_py310_sdkv2",
    conda_file="environment.yml",
    name="docker-image-plus-conda-example",
    description="Environment created from a Docker image plus Conda environment.",
    
)
ml_client.environments.create_or_update(env)


# Configure the scoring environment
inference_config = InferenceConfig(entry_script=script_file,
                                   environment=env)

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)

service_name = "croppredict-service3"

service = Model.deploy(ws, service_name, [model], inference_config, deployment_config)

service.wait_for_deployment(True)
print(service.state)

TypeError: object.__init__() takes exactly one argument (the instance to initialize)